In [ ]:
# Import Dependencies
import pandas as pd
import praw
import datetime as dt

In [10]:
# Define our custom functions
def get_date(created):
    return dt.datetime.fromtimestamp(created)

def mine_reddit(subreddit_name, keyword):
    subreddit = reddit.subreddit(subreddit_name).search(keyword, time_filter='all', limit=None)

    topics_dict = { "title":[], 
                    "score":[], 
                    "id":[], "url":[], 
                    "comms_num": [], 
                    "created": [], 
                    "body":[]}

    for submission in subreddit:
        topics_dict["title"].append(submission.title)
        topics_dict["score"].append(submission.score)
        topics_dict["id"].append(submission.id)
        topics_dict["url"].append(submission.url)
        topics_dict["comms_num"].append(submission.num_comments)
        topics_dict["created"].append(submission.created)
        topics_dict["body"].append(submission.selftext)

    topics_data = pd.DataFrame(topics_dict)

    _timestamp = topics_data["created"].apply(get_date)

    topics_data = topics_data.assign(timestamp = _timestamp)

    topics_data = topics_data[['title', 'score', 'comms_num', 'body', 'timestamp']]
    topics_data.rename({'timestamp':'Date'}, axis=1, inplace=True)
    topics_data.loc[:, 'Date'] = pd.to_datetime(topics_data.loc[:, 'Date'].apply(lambda x: str(x)[0:10]))
    
    return topics_data

In [6]:
# Configurate praw module
reddit = praw.Reddit(client_id='NLNG5cXR5iRB_A', 
                     client_secret='GALvnlicLlGbr80Ohie86Cy-nUPM3A', 
                     user_agent='scrapper', 
                     username='py_scrapper', 
                     password='Yy@5129468')

# Searching Tesla related post inside the StockMarket sub
subreddit_name='StockMarket'
keyword=['Tesla', 'TSLA', 'tsla']
scrapped = {}
for k in keyword:
    scrapped[f'{k}'] = mine_reddit(subreddit_name, keyword)

# Concatenate and save our results
tesla_reddit = pd.concat(scrapped)
tesla_reddit.to_csv('reddit/tesla_reddit.csv', index=False)

In [9]:
# Check the final result
tesla_reddit

title  score  \
Tesla 0                                 TSLA - time to VOTE!   2534   
      1                       TSLA will continue to go down?    428   
      2    TSLA: Post your projected market close for thi...    484   
      3    Market slightly down overall, but TSLA's on an...    698   
      4    Deutsche bank raises $TSLA PT to $1,200 from $...     63   
...                                                      ...    ...   
tsla  236  Wall Street Week Ahead for the trading week be...      8   
      237                        $TSLA Long Tesla above $247      1   
      238  I scraped r/stockmarket for the top ticker men...      5   
      239                            $30K Challenge - Update    245   
      240            TSLA smack down gathered some speed -5%     41   

           comms_num                                               body  \
Tesla 0          349                                                      
      1          297  Elon musk said he will sell 10% of his TSLA sh...   
      2          206                                                      
      3          166                                                      
      4           45                                                      
...              ...                                                ...   
tsla  236          6  Good Saturday morning to all of you here on r/...   
      237          8  [https://www.tradingview.com/chart/TSLA/X61Fxm...   
      238          7  &#x200B;\n        \n|Ticker|Company|# Mentions...   
      239         35  **The Challenge:** *Double a $30K account in f...   
      240         29  It seems like a big hit over the past two days...   

                Date  
Tesla 0   2021-11-07  
      1   2021-11-15  
      2   2021-11-08  
      3   2021-11-10  
      4   2021-12-31  
...              ...  
tsla  236 2021-11-13  
      237 2019-10-10  
      238 2021-11-12  
      239 2021-08-07  
      240 2017-07-07  

[723 rows x 5 columns]